### Compute RSA matrices by Odor btw encoding and retrieval
TF from 0 to 2s post odor 

In [13]:
from RSA import _btw_rsa_twoconds
import numpy as np
from itertools import product
from os.path import isfile, join, exists
from brainpipe.system import study
from joblib import Parallel, delayed

In [15]:
"""
Compute RSA matrices btw Encoding and Retrieval by Odor and ROI
Possibility to average RSA matrix across trials OR not
IMPORTANT for statistics to have all trials
"""

st = study('Olfacto')
path_TF = join(st.path,'feature/1_TF_{}_single_trial_rsa_by_odor/')
save_path = join(st.path,'feature/1_RSA_E_R_by_odor/')
dataname = '{}_odor_{}_{}_theta_TF.npz'

rois_to_select = ['IFG','OFC','HC']
su_odor_groups = {
    'VACJ' : {'low':['11','14','12','10'],
              'high':['15','17','16','13']},
    'SEMC' : {'low':['7','10','11','12','13'],
              'high':['5','8','9']},
    'PIRJ' : {'low':['1','9','5'],
              'high':['4','6','7','18']},
    'LEFC' : {'low':['15','2','1','16'],
              'high':['14','3','4','17']},
    'FERJ' : {'low':['7','2','16','17'],
              'high':['12','1','5','13']},}

def rsa_E_R_iEEG(su,cond,od,roi,average):
    savename = save_path+su+'_'+cond+'_odor_'+od+'_'+roi+'_E_R_rsa_theta_avg={}.npz'.format(average)
    if not exists(savename):
        print('processing rsa for ', su, cond, od, roi)
        #load data from E and R (nfreqs nelecs nsamples ntrials)
        matE = np.load(path_TF.format('E')+dataname.format(su,od,roi))
        channelsE, xyzE, dataE = matE['channels'], matE['xyz_sel'], matE['TF']
        #print('TF E ',dataE.shape)
        matR = np.load(path_TF.format('R')+dataname.format(su,od,roi))
        channelsR, xyzR, dataR = matR['channels'], matR['xyz_sel'], matR['TF']
        #print('TF R ',dataR.shape)

        #SELECT ONLY CORRESPONDING ELECTRODES
        high_l = channelsE if len(channelsE) >= len(channelsR) else channelsR
        low_l = channelsE if len(channelsE) < len(channelsR) else channelsR
        ids = [i for i in high_l if i in low_l]
        nelecs, nsamples = len(ids), dataE.shape[2]

        #PREPARE COMMON DATA FOR RSA 
        ID_E = [i for i, el in enumerate(channelsE) if el in ids]
        ID_R = [i for i, el in enumerate(channelsR) if el in ids]
        channelsE_, xyzE_, dataE_ = channelsE[ID_E], xyzE[ID_E], dataE[:,ID_E]
        channelsR_, xyzR_, dataR_ = channelsR[ID_R], xyzR[ID_R], dataR[:,ID_R]
        print(dataE_.shape, dataR_.shape)
        
        #COMPUTE AND SAVE RSA
        if average == True:
            rsa_mat = np.zeros((nelecs,nsamples,nsamples))
        elif average == False:
            ncomb = dataE_.shape[-1]*dataR_.shape[-1]
            rsa_mat = np.zeros((nelecs,ncomb,nsamples,nsamples))
        for elec in range(nelecs):
            rsa_mat[elec] += _btw_rsa_twoconds(dataE_[:,elec],dataR_[:,elec],
                                    stat='pearson',average=average)

        kwargs = {}
        kwargs['channels'], kwargs['xyz'] = channelsE_, xyzE_
        kwargs['rsa'] = rsa_mat
        np.savez(savename,**kwargs)
        print('E data', dataE.shape, 'R data',dataR.shape)
        print('check rsa', rsa_mat.shape)
    else :
        print('RSA already computed --> ',savename)

average = False #average rsa matrix across trials
for su in su_odor_groups:
    for cond in su_odor_groups[su]:
        for od in su_odor_groups[su][cond]:
            Parallel(n_jobs=3)(delayed(
                    rsa_E_R_iEEG)(su,cond,od,roi,average) for roi in rois_to_select)

-> Olfacto loaded
